# EDA RFMiD

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import os
import progressbar

## Getting Data

In [1]:
RFMID_PATH = "./Data/Raw/VariedDrAndres/Messidor/BD1"

In [2]:
DB_PATH = './RAW_DATA'
if os.path.exists(DB_PATH):
    !rm -r {DB_PATH}
    os.mkdir(DB_PATH)
else:
    os.mkdir(DB_PATH)

In [4]:
def unzip_set(zipFile: str, outputPath: str = DB_PATH):
    print(zipFile)
    zipFile = os.path.sep.join([RFMID_PATH, zipFile])
    !7z x {zipFile} -o{outputPath}

In [5]:
[unzip_set(z) for z in os.listdir(RFMID_PATH) if '.zip' in z]

Base34.zip

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz (806C1),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./Data/Raw/VariedDrAndres/Messidor/BD1                                                1 file, 427105132 bytes (408 MiB)

Extracting archive: ./Data/Raw/VariedDrAndres/Messidor/BD1/Base34.zip
--
Path = ./Data/Raw/VariedDrAndres/Messidor/BD1/Base34.zip
Type = zip
Physical Size = 427105132

      0% - 20051208_42322_0400_PP.ti                                  1% 1 - 20051208_42546_0400_PP.ti                                    2% 2 - 20051208_42552_0400_PP.ti                                    3% 2 - 20051208_42552_0400_PP.ti                                    3% 3 - 20051209_38404_0400_PP.ti                                    4% 4 - 20051212_36525_0400_PP.ti                                    6% 5 - 20051212_36548_0400_PP.ti 

[None, None, None, None, None, None, None, None, None, None, None, None]

In [ ]:
!mv ./RAW_DATA/Base11/* ./RAW_DATA

In [9]:
!mv ./RAW_DATA/Base13/* ./RAW_DATA

In [10]:

!rm -r ./RAW_DATA/Base11
!rm -r ./RAW_DATA/Base13


rm: cannot remove './RAW_DATA/Base11': No such file or directory


In [19]:
for f in os.listdir(DB_PATH):
    !mv {os.path.sep.join([DB_PATH, f])}/* {DB_PATH}
    !rm -r {os.path.sep.join([DB_PATH, f])}

## Exploring the dataset

In [20]:
csv_path = os.path.sep.join([DB_PATH, 'RFMiD_Training_Labels.csv'])
training_df = pd.read_csv(csv_path)

training_df.dropna(how='all', axis=0, inplace=True)

# get ID + specified disease column
dr_training_df = training_df.loc[:, training_df.columns.isin(['ID', 'DR'])]

In [21]:
dr_training_df.groupby('DR')['ID'].count()

DR
0    1544
1     376
Name: ID, dtype: int64

In [22]:
balanced = dr_training_df.groupby('DR').sample(376)

balanced.groupby('DR')['ID'].count()

DR
0    376
1    376
Name: ID, dtype: int64

In [23]:
DATASET_PATH = './DB'
if os.path.exists(DATASET_PATH):
    !rm -r {DATASET_PATH}
    os.mkdir(DATASET_PATH)
else:
    os.mkdir(DATASET_PATH)

## Restructuring the dataset

In [24]:
dir_structure = {f'{i}':list(balanced.loc[balanced['DR'] == i].pop("ID")) for i in range(2)}

for k in dir_structure.keys():
        set_path = os.path.sep.join([DATASET_PATH, 'Training'])

        if not os.path.exists(set_path):
            os.mkdir(set_path)

        class_path = os.path.sep.join([set_path, k])

        if not os.path.exists(class_path):
            os.mkdir(class_path)
        
        widgets = [
            f"Building Dataset {'Training'}/{k}: ",
            progressbar.Percentage(),
            " ",
            progressbar.Bar(),
            " ",
            progressbar.ETA()
        ]

        
        for i, v in enumerate(dir_structure[k]):


            pbar = progressbar.ProgressBar(
                maxval=len(dir_structure[k]),
                widgets=widgets
            ).start()

            destination = os.path.sep.join([class_path, f'{v}.png'])
            origin = os.path.sep.join([DB_PATH, 'Training', f'{v}.png'])
            !mv {origin} {destination}

            pbar.update(i)

        pbar.finish()

Building Dataset Training/0: 100% |#############################| Time: 0:00:00
Building Dataset Training/1: 100% |#############################| Time: 0:00:00


In [25]:
def restructure_set(csv_file: str, folder: str, column: str):

    df_columns = ["ID"]
    df_columns.append(column)

    # read csv file from the DB path
    csv_path = os.path.sep.join([DB_PATH, csv_file])
    df = pd.read_csv(csv_path)
    
    df.dropna(how='all', axis=0, inplace=True)

    # get ID + specified disease column
    dr_df = df.loc[:, df.columns.isin(['ID', 'DR'])]
    
    # dictionary column:[ID]
    dir_structure = {f'{i}':list(dr_df.loc[dr_df[column] == i].pop("ID")) for i in range(2)}

    # creating a folder named as {column} containing all files in [ID]
    for k in dir_structure.keys():
        set_path = os.path.sep.join([DATASET_PATH, folder])

        if not os.path.exists(set_path):
            os.mkdir(set_path)

        class_path = os.path.sep.join([set_path, k])

        if not os.path.exists(class_path):
            os.mkdir(class_path)
        
        widgets = [
            f"Building Dataset {folder}/{k}: ",
            progressbar.Percentage(),
            " ",
            progressbar.Bar(),
            " ",
            progressbar.ETA()
        ]

        
        for i, v in enumerate(dir_structure[k]):


            pbar = progressbar.ProgressBar(
                maxval=len(dir_structure[k]),
                widgets=widgets
            ).start()

            destination = os.path.sep.join([class_path, f'{v}.png'])
            origin = os.path.sep.join([DB_PATH, folder, f'{v}.png'])
            !mv {origin} {destination}

            pbar.update(i)

        pbar.finish()

In [26]:
not_training = [f for f in os.listdir(DB_PATH) if 'Training' not in f]
csv_files = [f for f in not_training if '.csv' in f]
folders = [f for f in not_training if f not in csv_files]

for csv in csv_files:
    for f in folders:
        if f in csv: # folder name is included in csv filename, which means folder and csv file match
            restructure_set(csv, f, 'DR')

Building Dataset Validation/0: 100% |###########################| Time: 0:00:00
Building Dataset Validation/1: 100% |###########################| Time: 0:00:00
Building Dataset Test/0: 100% |#################################| Time: 0:00:00
Building Dataset Test/1: 100% |#################################| Time: 0:00:00
